# How to append workgraph
## Introduction
There are two ways to combine workgraphs:
- Use the workgraph as a node (group) inside another workgraph. 
- Append the workgraph directly to another workgraph.

This tutorial will show an example for the second case.



In [1]:
%load_ext aiida
from aiida import load_profile
load_profile()

Profile<uuid='57ccbf7d9e2b41b39edb2bfdaf725feb' name='default'>

## Relax, Bands and PDOS
Here is an example to combine `bands` workgraph, `pdos` workgraph with a `relax` node to form a new workgraph.

### Bands workgraph

In [2]:
from aiida_quantumespresso.workgraphs.bands_group import bands_workgraph
from aiida import load_profile
load_profile()

bands_wt = bands_workgraph(run_relax=False)
bands_wt

NodeGraphWidget(value={'name': 'WorkGraph', 'uuid': 'fb426c7e-fa8c-11ee-9412-02425c27bd68', 'state': 'CREATED',…

### PDOS workgraph

In [3]:
from aiida_quantumespresso.workgraphs.pdos_group import pdos_workgraph
pdos_wt = pdos_workgraph(run_scf=True)
pdos_wt

NodeGraphWidget(value={'name': 'WorkGraph', 'uuid': '10aaac7a-fa8d-11ee-9412-02425c27bd68', 'state': 'CREATED',…

### Append workgraph

First, create a workgraph with `relax` node:

In [4]:
from aiida_workgraph import WorkGraph, build_node

PwRelaxChainNode = build_node('aiida_quantumespresso.workflows.pw.relax.PwRelaxWorkChain')

wt = WorkGraph('Electronic Structure')
relax_node = wt.nodes.new(PwRelaxChainNode, name='relax')
wt

NodeGraphWidget(value={'name': 'Electronic Structure', 'uuid': '1ff27140-fa8d-11ee-9412-02425c27bd68', 'state'…

Now let's combine `bands` workgraph, `pdos` workgraph with a `relax` node to form a new workgraph.

To append a workgraph into another workgraph, there are two steps:
- append the workgraph
- adjust the links, add new links between nodes, and remove old links if needed.



In [5]:
# append a wroktree
wt.append(bands_wt, prefix='bands_')
wt.append(pdos_wt, prefix='pdos_')
# adjust the links
wt.links.new(relax_node.outputs['output_structure'], wt.nodes['bands_scf'].inputs['pw.structure'])
wt.links.new(relax_node.outputs['output_structure'], wt.nodes['pdos_scf'].inputs['pw.structure'])
wt

NodeGraphWidget(value={'name': 'Electronic Structure', 'uuid': '1ff27140-fa8d-11ee-9412-02425c27bd68', 'state'…

## Node group
One can compare it with the other method: using the workgraph as a node inside another workgraph.



In [9]:
wt = WorkGraph('Electronic Structure')
relax_node = wt.nodes.new(PwRelaxChainNode, name='relax')
bands_job = wt.nodes.new(bands_workgraph, name='bands_group')
pdos_job = wt.nodes.new(pdos_workgraph, name='pdos_group')
wt.links.new(relax_node.outputs['output_structure'], bands_job.inputs['structure'])
wt.links.new(relax_node.outputs['output_structure'], pdos_job.inputs['structure'])
wt

NodeGraphWidget(value={'name': 'Electronic Structure', 'uuid': 'f959a82c-fa8d-11ee-9412-02425c27bd68', 'state'…

### Pros
Show the top level workflow.
### Cons
Only know the exact node graph at runtime.